In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def read_excel(path):
    return pd.read_excel(path)

def imputing_data_row(df,missing):
    df_a = df[df['WAKTU'] == missing['WAKTU'] + pd.Timedelta(minutes=-15)]
    df_b = df[df['WAKTU'] == missing['WAKTU'] + pd.Timedelta(minutes=15)]
    MOTOR = -1
    MOBIL = -1
    TRUK = -1
    if(len(df_a) + len(df_b) == 2):
        result = pd.concat([df_a,df_b])
        MOTOR = round(result['MOTOR'].mean())
        MOBIL = round(result['MOBIL'].mean())
        TRUK = round(result['TRUK/BUS'].mean())
    return MOTOR, MOBIL, TRUK 

def imputing_data(df, HARI, JAM):
    df = df[(df['HARI'] == HARI) & (df['JAM'] == JAM)]
    
    # Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
    Q1 = df['SMP'].quantile(0.25)
    Q3 = df['SMP'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Menentukan batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Menemukan dan menghapus outlier
    df_cleaned = df[(df['SMP'] >= lower_bound) & (df['SMP'] <= upper_bound)]

    df_grouped = df_cleaned[['HARI', 'JAM', 'MOTOR', 'MOBIL','TRUK/BUS']]
    df_grouped = round(df_grouped.groupby(['HARI', 'JAM']).mean()).reset_index()

    return df_grouped

def perhitungan_jumlah_smp(df):
    df['SMP_MTR'] = df['MOTOR'] * 0.8
    df['SMP_MBL'] = df['MOBIL']
    df['SMP_TRK'] = df['TRUK/BUS'] * 2.5
    df['SMP'] = round(df['SMP_MTR'] + df['SMP_MBL'] + df['SMP_TRK'])
    df['JUMLAH'] = df['MOTOR'] + df['MOBIL'] + df['TRUK/BUS']
    return df

def menggabungkan_data(df, missing_times):
    # Menggabungkan data hilang dengan DataFrame utama
    df = pd.concat([df, missing_times], ignore_index=True)
    
    # Mengurutkan DataFrame berdasarkan kolom 'Waktu'
    df = df.sort_values(by=['TAHUN','BULAN','TANGGAL','HARI','JAM']).reset_index(drop=True)

    return df


In [3]:
path_1 = '../Data/Preprocessing/Data Losarang/2022/1_input_losarang_2022.xlsx'
path_2 = '../Data/Preprocessing/Data Losarang/2022/1_missing_times_losarang_2022.xlsx'

df = read_excel(path_1)
df_missing = read_excel(path_2)

df_missing.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2022-01-04 19:15:00,2022,1,1,4,Tuesday,19:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-04 19:30:00,2022,1,1,4,Tuesday,19:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-11 18:30:00,2022,1,2,11,Tuesday,18:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-11 18:45:00,2022,1,2,11,Tuesday,18:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-11 19:00:00,2022,1,2,11,Tuesday,19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for i, row in df_missing.iterrows():
    imputasi = imputing_data(df, row['HARI'], row['JAM'])
    break

In [5]:
imputasi

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS
0,Tuesday,19:15,93.0,66.0,39.0


In [6]:
# Iterasi setiap baris dan mengisi nilai yang hilang
for i, missing in df_missing.iterrows():
    MOTOR, MOBIL, TRUK = imputing_data_row(df, missing)
    if (MOTOR != -1) & (np.isnan(missing['MOTOR'])):
        df_missing.at[i, 'MOTOR'] = MOTOR
        df_missing.at[i, 'MOBIL'] = MOBIL
        df_missing.at[i, 'TRUK/BUS'] = TRUK

In [7]:
# Iterasi setiap baris dan mengisi nilai yang hilang
for i, missing in df_missing.iterrows():
    imputasi = imputing_data(df, missing['HARI'], missing['JAM'])
    if (not imputasi.empty) & (np.isnan(missing['MOTOR'])):
        df_missing.at[i, 'MOTOR'] = imputasi['MOTOR'].values[0]
        df_missing.at[i, 'MOBIL'] = imputasi['MOBIL'].values[0]
        df_missing.at[i, 'TRUK/BUS'] = imputasi['TRUK/BUS'].values[0]

In [8]:
df_missing = perhitungan_jumlah_smp(df_missing)

In [9]:
df_missing.to_excel('../Data/Preprocessing/Data Losarang/2022/2_missing_times_losarang_2022.xlsx', index=False)

In [10]:
df_missing.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2022-01-04 19:15:00,2022,1,1,4,Tuesday,19:15,93.0,66.0,39.0,198.0,74.4,66.0,97.5,238.0
1,2022-01-04 19:30:00,2022,1,1,4,Tuesday,19:30,83.0,55.0,41.0,179.0,66.4,55.0,102.5,224.0
2,2022-01-11 18:30:00,2022,1,2,11,Tuesday,18:30,117.0,67.0,45.0,229.0,93.6,67.0,112.5,273.0
3,2022-01-11 18:45:00,2022,1,2,11,Tuesday,18:45,101.0,64.0,41.0,206.0,80.8,64.0,102.5,247.0
4,2022-01-11 19:00:00,2022,1,2,11,Tuesday,19:00,98.0,61.0,42.0,201.0,78.4,61.0,105.0,244.0


In [11]:
df_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5147 entries, 0 to 5146
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   WAKTU     5147 non-null   datetime64[ns]
 1   TAHUN     5147 non-null   int64         
 2   BULAN     5147 non-null   int64         
 3   PEKAN     5147 non-null   int64         
 4   TANGGAL   5147 non-null   int64         
 5   HARI      5147 non-null   object        
 6   JAM       5147 non-null   object        
 7   MOTOR     5147 non-null   float64       
 8   MOBIL     5147 non-null   float64       
 9   TRUK/BUS  5147 non-null   float64       
 10  JUMLAH    5147 non-null   float64       
 11  SMP_MTR   5147 non-null   float64       
 12  SMP_MBL   5147 non-null   float64       
 13  SMP_TRK   5147 non-null   float64       
 14  SMP       5147 non-null   float64       
dtypes: datetime64[ns](1), float64(8), int64(4), object(2)
memory usage: 603.3+ KB


In [12]:
df_compiled = menggabungkan_data(df, df_missing)
df_compiled.head()

,WAKTU,TAHUN,BULAN,PEKAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2022-01-01 00:00:00,2022,1,1,1,Saturday,00:00,185.0,39.0,24.0,248.0,148.0,39.0,60.0,247.0
1,2022-01-01 00:15:00,2022,1,1,1,Saturday,00:15,195.0,64.0,35.0,294.0,156.0,64.0,87.5,308.0
2,2022-01-01 00:30:00,2022,1,1,1,Saturday,00:30,194.0,49.0,17.0,260.0,155.2,49.0,42.5,247.0
3,2022-01-01 00:45:00,2022,1,1,1,Saturday,00:45,162.0,43.0,21.0,226.0,129.6,43.0,52.5,225.0
4,2022-01-01 01:00:00,2022,1,1,1,Saturday,01:00,125.0,58.0,18.0,201.0,100.0,58.0,45.0,203.0


In [13]:
df_compiled.to_excel('../Data/Preprocessing/Data Losarang/2022/2_input_losarang_2022.xlsx', index=False)

In [14]:
len(df_compiled)

35040